# Risk Attribution: Menchero Methodology ###

In [1]:
# Import Libraries

# Data Management
import pandas as pd
import numpy as np

# Plots
import matplotlib.pyplot as plt

# Handle Files
import sys
import os

# Import Local Functions
sys.path.append(os.path.abspath("../source"))
from config import get_tickers
from factors_toolkit import FamaFrenchFactors
from portfolios_toolkit import markowitz_weights
from portfolios_toolkit import portfolio_variance

### Building a Portfolio ###

In [2]:
tickers = get_tickers("6.3")

tickers

In [3]:
# Import the Returns Data
returns_df = pd.read_csv(r'..\additional_data\stocks_returns.csv')
returns_df.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
returns_df.set_index('Date', inplace=True)
returns_df.index = pd.to_datetime(returns_df.index)

returns_df

In [4]:
# Make the portfolio DataFrame
portfolio_holdings_df = returns_df[tickers]
portfolio_holdings_df = portfolio_holdings_df

portfolio_holdings_df

In [5]:
# Portfolio's Returns and Variances
expected_returns = portfolio_holdings_df.mean()
covariance_matrix = portfolio_holdings_df.cov()

In [6]:
# Obtain Weights
p_weights = markowitz_weights(
    expected_returns, 
    covariance_matrix,
    0.0015
)

p_weights

In [7]:
# Create a Portfolio Weights DF
portfolio_weights = pd.Series(
    p_weights,
    index = portfolio_holdings_df.columns,
    name = 'weights'
)

portfolio_weights

In [8]:
# Portfolios Variance
p_variance = portfolio_variance(p_weights, portfolio_holdings_df)

p_variance

In [9]:
# Portfolio Returns
portfolio_returns = portfolio_holdings_df @ p_weights
portfolio_returns.name = 'portfolio_returns'

In [10]:
# Holdings Standard Deviations
holdings_stds = portfolio_holdings_df.std()

holdings_stds

In [11]:
# Holding's correlations with the portfolio
corr_with_portfolio = portfolio_holdings_df.corrwith(portfolio_returns)

corr_with_portfolio

The reader can review the derivation of the x-sigma-rho methodology in Section 3 of this module’s PDF.

In [12]:
# Now get the x-sigma-rho
RC = portfolio_weights * holdings_stds * corr_with_portfolio
RC.name = 'risk_contribution'

RC

In [13]:
RC.sum()

In [14]:
np.sqrt(p_variance)

In [14]:
# In percentage
RC_percent = (RC / np.sqrt(p_variance)) * 100

RC_percent

In [15]:
# Create Plot
plt.figure(figsize=(10, 6))
RC_percent.plot(kind='bar', label='Risk Contribution')

# Config
plt.title('Percentage Risk Contribution')
plt.xlabel('Stock')
plt.ylabel('Percentage')
plt.legend()
plt.grid()

# Show
plt.show() 

In [16]:
# Marginal Risk Contribution
MRC = holdings_stds * corr_with_portfolio
MRC.name = 'marginal_risk_contribution'

MRC

In [17]:
# Create Plot
plt.figure(figsize=(10, 6))
MRC.plot(kind='bar', label='Marginal Risk Contribution')

# Config
plt.title('Marginal Risk Contribution')
plt.xlabel('Stock')
plt.ylabel('Percentage')
plt.legend()
plt.grid()

# Show
plt.show() 

We conclude that our portfolio’s risk is more sensitive to NVDA’s exposure than to that of any other stock in the portfolio.

### Now Using Factor ###

In [15]:
# Now import the premiums
premiums_df = pd.read_csv(r'..\additional_data\famafrench_premiums.csv')
premiums_df.set_index('Date', inplace=True)
premiums_df.index = pd.to_datetime(premiums_df.index)
premiums_df.columns = ['mkt_premium', 'smb_premium', 'hml_premium', 'risk_free_rate']
premiums_df = premiums_df.div(100)

premiums_df

In [16]:
# Now calculate the betas
portfolio_premium = portfolio_returns - premiums_df['risk_free_rate']

# Check if the Function Works
parameters = FamaFrenchFactors(
    portfolio_premium,
    premiums_df['mkt_premium'],
    premiums_df['smb_premium'],
    premiums_df['hml_premium'],
)

parameters

In [17]:
# Create the Parameters Series
portfolio_parameters = pd.DataFrame([parameters])
portfolio_parameters.columns = ['const', 'mkt_premium', 'smb_premium', 'hml_premium']
portfolio_parameters = portfolio_parameters.T
portfolio_parameters.columns = ['parameters']
portfolio_parameters = portfolio_parameters['parameters']
portfolio_parameters = portfolio_parameters.iloc[1:]

portfolio_parameters

In [18]:
# Get the standard deviations of the premiums
premiums_std = premiums_df[['mkt_premium', 'smb_premium', 'hml_premium']].std()
premiums_std.name = 'premium_std'

premiums_std

In [19]:
# Holding's correlations with the portfolio
factors_corr_with_portfolio = premiums_df[['mkt_premium', 'smb_premium', 'hml_premium']].corrwith(portfolio_returns)

factors_corr_with_portfolio

In [20]:
# Now get the x-sigma-rho
RC = portfolio_parameters * premiums_std * factors_corr_with_portfolio
RC.name = 'risk_contribution'

RC

In [21]:
RC.sum()

In [26]:
np.sqrt(portfolio_returns.var()) - RC.sum()

In [ ]:
# Para calcular el riesgo idio:

# 1) Calcular los Retornos Idio (Retorno de las STOCKS - Xf) u = retorno especifico
# 2) A la serie de tiempo de u (std en rolling window) 126 - 252 días 
# 3) Con la misma ventana de tiempo y mismo half life se hace un covariance (matrix de covariance de factores)

In [24]:
# In percentage
RC_percent = (RC / np.sqrt(p_variance)) * 100

RC_percent

In [25]:
# Idiosyncratic Risk
idio_risk = 100 - RC_percent.sum()
idio_risk

### Correlation Drilldown ###

In Section 3 of the PDF, the correlation drilldown is explored in greater depth and with more mathematical rigor

In [21]:
# Calculate the Sigmas
sigmas = portfolio_holdings_df.std()

sigmas

In [22]:
# The Covariances
gammas = portfolio_holdings_df.cov()

gammas

In [29]:
# Portfolio Sigma
sigma_P = np.sqrt(p_variance)

sigma_P

In [30]:
# The Correlations
rhos = portfolio_holdings_df.corr()

rhos

In [53]:
# Let us check the drilldowns
decomp = pd.DataFrame(index=portfolio_holdings_df.columns, columns=portfolio_holdings_df.columns)

# Loop
for m in portfolio_holdings_df.columns:  # Stock m (rows)
    for n in portfolio_holdings_df.columns:  # Stock n (columns)
        
        term = portfolio_weights.loc[n] * (sigmas.loc[n] / sigma_P) * rhos.loc[m, n]
        
        decomp.loc[m, n] = term

decomp

In [55]:
# If we sum horizontally
rho_m_R = decomp.sum(axis=1)
rho_m_R.name = 'rho_m_R'

rho_m_R

In [56]:
# Check that both ways to calculate the correlation with the portfolio are valid
corr_with_portfolio

In [58]:
# Now let us discount the self-interaction factor from the correlations:
rho_m_R_discounted = rho_m_R - decomp.values.diagonal()
rho_m_R_discounted.name = 'rho_m_R_discounted'

rho_m_R_discounted